In [1]:
import json
import numpy as np
import pandas as pd
from sys import exit, stderr, stdin, stdout
from time import sleep
# from tqdm import tqdm
from tqdm.notebook import tqdm
from os.path import dirname, join, isfile, isdir
from os import makedirs
import subprocess
from datetime import datetime, timedelta, date

from bulletin.data.notifica import Notifica
from bulletin.commom.utils import Timer, get_better_notifica
from bulletin.commom.normalize import normalize_hash
from bulletin.commom.static import meses
from bulletin.metabase.request import download_metabase

pd.set_option('display.max_columns', None)

output = join("output","correcoes","notifica")

if not isdir(output):
    makedirs(output)

tqdm.pandas()

timerAll = Timer()
timer = Timer()

In [2]:
baixar_notifica = False#True if input("Enter para continuar, S para baixar notifica") == 'S' else False
ler_notifica = False#True if input("Enter para continuar, S para ler notifica") == 'S' else False

In [3]:
notifica = Notifica()
if ler_notifica or baixar_notifica:
    if baixar_notifica:
        notifica.download_todas_notificacoes()
    if ler_notifica:
        notifica.read_todas_notificacoes()
        notifica.save()
else:
    notifica.load()

print(notifica.shape())

(2817308, 1290846, 15638, 1482354, 2443)


In [4]:
casosn = notifica.get_casos()
# casosn = casosn.loc[(casosn['mun_resid']=='CURITIBA') & (casosn['classificacao_final']==2)]
casosn.groupby('classificacao_final')[['id']].count().append(pd.DataFrame(index=['total'],columns=['id'],data=[len(casosn)])).to_csv('classificacao_final_count.csv')

,id
2,142895
total,142895


In [5]:
timer.start()

pacientes = { 'pacientes': casosn['paciente'].apply(normalize_hash).values.tolist() }
with open('pacientes.json', 'w') as json_file:
  json.dump(pacientes, json_file)

subprocess.call(("node", 'soundex.js'))

with open('pacientes_soundex.json', 'r') as json_file:
  pacientes = json.load(json_file)

casosn['soundex'] = pacientes
casosn = casosn.sort_values('soundex')

timer.stop()
print('soundex: ',timer.ftime())

142895


,soundex,paciente
1280859,1SRDRGSDZV,LHAIS RODRIGUES DA SILVA
1746721,21NRKRBSMSD,RAUL HENRIQUE RIBAS MACEDO
1763050,21RSLVNTR,RAUL HERCILIO VENTURA
1685286,21RSLVNTRJN2,RAUL HERCILIO VENTURA JUNIOR
1026569,23DSTFNSJN2,REINHOLD STEPHANES JUNIOR
1724455,23NMLRBR,RAYNHANA MAELLY RIBEIRO
1214751,23NRKDZV,RUAN HENRIQUE DA SILVA
1154923,23NRKKRPMNSPR,RYAN HENRIQUE KROPMANNS PIERI
1734250,23NRKKRPMNSPR,RYAN HENRIQUE KROPMANNS PIERI
1128058,23NRKRDRGSDZV,RYAN HENRYQUE RODRIGUES DA SILVA


In [6]:
timer.start()

keys = ['soundex','paciente','idade','ibge_unidade_notifica','ibge_residencia','data_nascimento','nome_mae','cpf','id']

casosn_hash = casosn[keys+['status_notificacao','classificacao_final','metodo','evolucao']]
casos_agrupados = casosn_hash.sort_values('soundex').groupby(keys[:-1])
casos_agrupados_count = casos_agrupados[['id']].count().rename(columns={'id':'count'}).sort_values('count', ascending=False)
casos_agrupados_count.to_csv('casos_agrupados_count.csv')

timer.stop()
print('casos_agrupados_count: ',timer.ftime())

00:00:00.551


,,,,,,,,count
soundex,paciente,idade,ibge_unidade_notifica,ibge_residencia,data_nascimento,nome_mae,cpf,
JVKTRDMBKRJ,JOAO VICTOR DEMBICKI ARAUJO,28,410420,410690,1992-05-05,ELIANE APARECIDA DEMBICKI ARAUJO,07609193992,6
ELZSPK,ELOISE SPILKA,29,410690,410690,1991-07-04,ALVANEZ MARIOT SPILKA,06239631914,5
SMPRRDZVDLM,SIMEIA PEREIRA DA SILVA DE LIMA,39,410420,410690,1981-11-01,DELAIR DE JESUS ANDRADE SILVA,04862225942,5
MNLKNSRTR,MANUELLE QUINES SARTORI,28,410690,410690,1992-10-08,ADRIANE QUINES SARTORI,08805778940,5
JKLNBRKDSNTS,JAKLINE BROCO DOS SANTOS,30,410690,410690,1990-09-26,NILVA DE FATIMA BROCO,07407421919,5
...,...,...,...,...,...,...,...,...
FLPLVRPNTS,FELIPE OLIVEIRA PONTES,17,410690,410690,2003-02-17,EDILAINE DE OLIVEIRA,12921621983,1
FLPLVRDS,FELYP OLIVEIRA DIAS,27,410690,410690,1993-03-20,MARIA DA CONCEICAO OLIVEIRA DIAS,07959250936,1
FLPLTKRSTN,FELIPE LEITE CRISTINO,27,999999,410690,1993-06-20,GLACI LEITE CRISTINO,08596303901,1


In [11]:
timer.start()

pacientes = pd.DataFrame(index=casos_agrupados_count.index,columns=['ids','status_notificacao','classificacao_final','metodo','evolucao'], data=[])

for idx,df in casos_agrupados:
    pacientes.loc[idx,'ids'] = [ x for x in df['id'].values ]
    pacientes.loc[idx,'status_notificacao'] = [ x for x in df['status_notificacao'].values ]
    pacientes.loc[idx,'classificacao_final'] = [ x for x in df['classificacao_final'].values ]
    pacientes.loc[idx,'metodo'] = [ x for x in df['metodo'].values ]
    pacientes.loc[idx,'evolucao'] = [ x for x in df['evolucao'].values ]

pacientes.to_csv('pacientes.csv')
timer.stop()
print(f'pacientes ({len(pacientes)}): ',timer.ftime())

/home/vincslg/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:2894: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(
00:02:10.210


,,,,,,,,ids,status_notificacao,classificacao_final,metodo,evolucao
soundex,paciente,idade,ibge_unidade_notifica,ibge_residencia,data_nascimento,nome_mae,cpf,,,,,
JVKTRDMBKRJ,JOAO VICTOR DEMBICKI ARAUJO,28,410420,410690,1992-05-05,ELIANE APARECIDA DEMBICKI ARAUJO,07609193992,"[502441, 743165, 1032469, 1683854, 1508122, 12...","[3, 3, 3, 3, 2, 3]","[2, 2, 2, 2, 2, 2]","[3, 2, 2, 6, 2, 2]","[3, 1, 3, 3, 3, 1]"
ELZSPK,ELOISE SPILKA,29,410690,410690,1991-07-04,ALVANEZ MARIOT SPILKA,06239631914,"[2089325, 1299358, 2140794, 2065074, 1685532]","[3, 3, 2, 3, 3]","[2, 2, 2, 2, 2]","[1, 3, 1, 1, 1]","[3, 3, 3, 3, 3]"
SMPRRDZVDLM,SIMEIA PEREIRA DA SILVA DE LIMA,39,410420,410690,1981-11-01,DELAIR DE JESUS ANDRADE SILVA,04862225942,"[1956098, 1917232, 1987194, 1798516, 1875349]","[3, 3, 1, 3, 3]","[2, 2, 2, 2, 2]","[6, 6, 6, 6, 6]","[3, 3, 3, 3, 3]"
MNLKNSRTR,MANUELLE QUINES SARTORI,28,410690,410690,1992-10-08,ADRIANE QUINES SARTORI,08805778940,"[1669737, 2009732, 2104715, 1669154, 2104792]","[3, 3, 3, 3, 1]","[2, 2, 2, 2, 2]","[1, 1, 1, 1, 1]","[3, 3, 1, 1, 3]"
JKLNBRKDSNTS,JAKLINE BROCO DOS SANTOS,30,410690,410690,1990-09-26,NILVA DE FATIMA BROCO,07407421919,"[1333676, 1335606, 1694921, 1574803, 1531127]","[3, 3, 2, 3, 3]","[2, 2, 2, 2, 2]","[1, 3, 1, 1, 1]","[3, 3, 3, 3, 3]"
...,...,...,...,...,...,...,...,...,...,...,...,...
FLPLVRPNTS,FELIPE OLIVEIRA PONTES,17,410690,410690,2003-02-17,EDILAINE DE OLIVEIRA,12921621983,[1145528],[1],[2],[1],[3]
FLPLVRDS,FELYP OLIVEIRA DIAS,27,410690,410690,1993-03-20,MARIA DA CONCEICAO OLIVEIRA DIAS,07959250936,[524788],[1],[2],[1],[3]
FLPLTKRSTN,FELIPE LEITE CRISTINO,27,999999,410690,1993-06-20,GLACI LEITE CRISTINO,08596303901,[1655998],[1],[2],[1],[3]


In [8]:
timer.start()

pacientes_duplicados = pacientes.loc[ [True if len(ids) > 1 else False for ids in pacientes['ids']]]
pacientes_duplicados.to_csv('pacientes_duplicados.csv')

timer.stop()
print(f'pacientes_duplicados ({len(pacientes_duplicados)}): ',timer.ftime())

,,,,,,,,ids,status_notificacao,classificacao_final,metodo,evolucao
soundex,paciente,idade,ibge_unidade_notifica,ibge_residencia,data_nascimento,nome_mae,cpf,,,,,
JVKTRDMBKRJ,JOAO VICTOR DEMBICKI ARAUJO,28,410420,410690,1992-05-05,ELIANE APARECIDA DEMBICKI ARAUJO,07609193992,"[502441, 743165, 1032469, 1683854, 1508122, 12...","[3, 3, 3, 3, 2, 3]","[2, 2, 2, 2, 2, 2]","[3, 2, 2, 6, 2, 2]","[3, 1, 3, 3, 3, 1]"
ELZSPK,ELOISE SPILKA,29,410690,410690,1991-07-04,ALVANEZ MARIOT SPILKA,06239631914,"[2089325, 1299358, 2140794, 2065074, 1685532]","[3, 3, 2, 3, 3]","[2, 2, 2, 2, 2]","[1, 3, 1, 1, 1]","[3, 3, 3, 3, 3]"
SMPRRDZVDLM,SIMEIA PEREIRA DA SILVA DE LIMA,39,410420,410690,1981-11-01,DELAIR DE JESUS ANDRADE SILVA,04862225942,"[1956098, 1917232, 1987194, 1798516, 1875349]","[3, 3, 1, 3, 3]","[2, 2, 2, 2, 2]","[6, 6, 6, 6, 6]","[3, 3, 3, 3, 3]"
MNLKNSRTR,MANUELLE QUINES SARTORI,28,410690,410690,1992-10-08,ADRIANE QUINES SARTORI,08805778940,"[1669737, 2009732, 2104715, 1669154, 2104792]","[3, 3, 3, 3, 1]","[2, 2, 2, 2, 2]","[1, 1, 1, 1, 1]","[3, 3, 1, 1, 3]"
JKLNBRKDSNTS,JAKLINE BROCO DOS SANTOS,30,410690,410690,1990-09-26,NILVA DE FATIMA BROCO,07407421919,"[1333676, 1335606, 1694921, 1574803, 1531127]","[3, 3, 2, 3, 3]","[2, 2, 2, 2, 2]","[1, 3, 1, 1, 1]","[3, 3, 3, 3, 3]"
...,...,...,...,...,...,...,...,...,...,...,...,...
ASNDZNKDRM,ALCINDA SANCAO DARIN,82,410690,410690,1937-08-25,FRANCISCA DIAS DE ARRUDA,63151502920,"[580926, 555368]","[1, 3]","[2, 2]","[1, 1]","[2, 3]"
KLDMRF2RDSNTS,CLAUDIMIR FERREIRA DOS SANTOS,32,410690,410690,1988-08-02,ROSELI DE FATIMA FERREIRA DOS SANTOS,01043830979,"[1355505, 1417246]","[3, 1]","[2, 2]","[1, 1]","[3, 3]"
2BRTDZZDZV,ROBERTO DE SOUZA DA SILVA,56,410690,410690,1964-08-26,ANESIA DE SOUZA DA SILVA,65733142987,"[1418122, 1676855]","[2, 3]","[2, 2]","[1, 3]","[0, 3]"


In [ ]:
exit()
casos_duplicados = casosn.loc[(casosn['cpf'].notnull()) & (casosn.duplicated('cpf',keep=False))].sort_values('cpf')
casosn.loc[casos_duplicados.index, 'duplicado'] = 1 

print(f"Total de pacientes com mais de uma ocorrencia: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('cpf'))} pacientes pelo CPF que estavam com mais de uma ocorrencia")

all_casos_duplicados = set()
all_duplicados_mantidos = set()
manter = []

for hash, group in casos_duplicados.groupby('cpf'):
    idx = get_better_notifica(group)
    #check reinfection
    manter.append(idx)


casosn.loc[manter, 'manter'] = 1 

casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_cpf.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()
if dropar:
    casosn = casosn.drop(index=casos_duplicados)

In [ ]:
casos_duplicados = casosn.loc[(casosn['nome_mae'].notnull()) & (casosn.duplicated('hash_mae',keep=False))].sort_values('nome_mae')
casosn.loc[casos_duplicados.index, 'duplicado'] = 1 

print(f"Total de pacientes com mais de uma ocorrencia pelo nome+nome_mae: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('nome_mae'))} pacientes pelo nome+nome_mae que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('nome_mae'):
    idx = get_better_notifica(group)
    manter.append(idx)

casosn.loc[manter, 'manter'] = 1 

casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_mae.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()
if dropar:
    casosn = casosn.drop(index=casos_duplicados)

In [ ]:
casos_duplicados = casosn.loc[(casosn['hash_nasc'].notnull()) & (casosn.duplicated('hash_nasc',keep=False))].sort_values('data_nascimento')
casosn.loc[casos_duplicados.index, 'duplicado'] = 1 

print(f"Total de pacientes com mais de uma ocorrencia pelo data_nascimento: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_nasc'))} pacientes pelo data_nascimento que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_nasc'):
    idx = get_better_notifica(group)
    manter.append(idx)

casosn.loc[manter, 'manter'] = 1 

casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_data_nascimento.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()
if dropar:
    casosn = casosn.drop(index=casos_duplicados)

In [ ]:
casos_duplicados = casosn.loc[(casosn['hash_resid'].notnull()) & (casosn.duplicated('hash_resid',keep=False))].sort_values('paciente')
casosn.loc[casos_duplicados.index, 'duplicado'] = 1 

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_resid: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_resid'))} pacientes pelo hash_resid que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_resid'):
    idx = get_better_notifica(group)
    manter.append(idx)

casosn.loc[manter, 'manter'] = 1 

casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_idade_mun_resid.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()
if dropar:
    casosn = casosn.drop(index=casos_duplicados)

In [ ]:
casos_duplicados = casosn.loc[(casosn['hash_atend'].notnull()) & (casosn.duplicated('hash_atend',keep=False))].sort_values('paciente')
casosn.loc[casos_duplicados.index, 'duplicado'] = 1 

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_atend: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_atend'))} pacientes pelo hash_atend que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_atend'):
    idx = get_better_notifica(group)
    manter.append(idx)

casosn.loc[manter, 'manter'] = 1 

casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_nome_idade_mun_atend.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()
if dropar:
    casosn = casosn.drop(index=casos_duplicados)

In [ ]:
casos_duplicados = casosn.loc[(casosn['hash_diag'].notnull()) & (casosn.duplicated('hash_diag',keep=False))].sort_values('paciente')
casosn.loc[casos_duplicados.index, 'duplicado'] = 1 

print(f"Total de pacientes com mais de uma ocorrencia pelo hash_diag: {len(casos_duplicados)}")
print(f"{len(casos_duplicados.groupby('hash_diag'))} pacientes pelo hash_diag que estavam com mais de uma ocorrencia")

manter = []
for hash, group in casos_duplicados.groupby('hash_diag'):
    idx = get_better_notifica(group)
    manter.append(idx)

casosn.loc[manter, 'manter'] = 1 

casos_duplicados = set(casos_duplicados.index.tolist())
casos_duplicados = casos_duplicados - set(manter)

all_duplicados_mantidos |= set(manter)
all_casos_duplicados |= casos_duplicados
casosn.loc[casos_duplicados].to_csv(join(output,'pacientes_duplicados_hash_diag.csv'), index=False)

casosn.loc[casos_duplicados].groupby('classificacao_final')[['id']].count()
if dropar:
    casosn = casosn.drop(index=casos_duplicados)

In [ ]:
notifica.save(casosn)